In [327]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression, RFECV, SelectFromModel
from sklearn.linear_model import LinearRegression, SGDRegressor


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 80)

In [328]:
def gd_path(file_id):
    """Generate a shareable link from Google Drive file id."""
    return f"https://drive.google.com/uc?export=download&id={file_id}"

files_id = {
    'housing_data':"1kX_jcLeBpBGvTo8FXDeU2MK-aw1a0voU", #iteration7 data
    'test_housing_data':"1CMsAWhWKWBjI6DDEHtcYmRVZRazfE9bo", #test data for housing
    'ids_com':"10gwiL49calkj-xbx-3rQEK4H2zoJcU11" #ID for the commiting of the project

}

housing_data = pd.read_csv(gd_path(files_id['housing_data']), sep=",")
test_housing_data = pd.read_csv(gd_path(files_id['test_housing_data']), sep=",")
ids_com = pd.read_csv(gd_path(files_id['ids_com']), sep=",")
df = housing_data

# Pre-Processing Pipe

## Ordinary columns

This part was done by hands. Many Bothans died to bring us this information.

In [329]:
from sklearn.model_selection import train_test_split

# X and y creation
y = df.pop("SalePrice")

# Feature Engineering
X = df

# data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [332]:
# building the pipeline
X_cat = X.select_dtypes(exclude="number").copy()
X_num = X.select_dtypes(include="number").copy()

#  numerical pipeline
numeric_pipe = make_pipeline(
    SimpleImputer(strategy="mean", fill_value="N_A"),
    MinMaxScaler(),
    VarianceThreshold(threshold=0.02))# maybe remove it
    


# categorical pipeline
# defining ordinal & onehot columns

ordinal_cols = X_cat.columns.get_indexer(['LandContour', 'LandSlope',
                     'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
                     'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                     'HeatingQC', 'KitchenQual', 'Functional',
                     'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond',
                     'PoolQC']) 

onehot_cols = X_cat.columns.get_indexer(['MSZoning', 'Condition1', 'Heating', 'Street', 'CentralAir', 
                                         'Foundation', 'Alley', 'LotShape', 'Utilities', 'LotConfig', 
                                         'Neighborhood', 'Condition2', 'BldgType', 'HouseStyle', 
                                         'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 
                                         'Electrical', 'GarageType', 'PavedDrive', 'Fence', 'MiscFeature',
                                         'SaleType', 'SaleCondition'])



# defining the categorical encoder
# we manually establish the order of the categories for our ordinal feature (Cabin), including "N_A"
LandContour = ['Lvl', 'Bnk', 'HLS', 'Low', 'N_A']
LandSlope = ['Gtl', 'Mod', 'Sev', 'N_A']
ExterQual = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
ExterCond = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
BsmtQual = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
BsmtCond = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
BsmtExposure = ['Gd', 'Av', 'Mn', 'No', 'N_A']
BsmtFinType1 = ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'N_A']
BsmtFinType2 = ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'N_A']
HeatingQC = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
KitchenQual = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
Functional = ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal', 'N_A']
FireplaceQu = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
GarageFinish = ['Fin', 'RFn', 'Unf', 'N_A']
GarageQual = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
GarageCond = ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A']
PoolQC = ['Ex', 'Gd', 'TA', 'Fa', 'N_A']


categorical_encoder = ColumnTransformer(
    transformers=[
        ("cat_ordinal", OrdinalEncoder(categories=[LandContour,LandSlope,
                                                    ExterQual, ExterCond, BsmtQual, BsmtCond,
                                                    BsmtExposure, BsmtFinType1, BsmtFinType2,
                                                    HeatingQC, KitchenQual, Functional,
                                                    FireplaceQu, GarageFinish, GarageQual, GarageCond,
                                                    PoolQC]), ordinal_cols),

        ("cat_onehot", OneHotEncoder(handle_unknown="ignore"), onehot_cols)
    ]
)


categoric_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="N_A"),
    categorical_encoder 
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num_pipe", numeric_pipe, X_num.columns),
        ("cat_pipe", categoric_pipe, X_cat.columns),
    ]
)

## Collinearity

In [ ]:
corrMatrix = X_train.corr()

# Filter the correlation matrix to keep only values above 0.5
corrMatrix_filtered = corrMatrix[corrMatrix > 0.6]

# Set the size of the heatmap to be 18x18 inches
plt.figure(figsize=(18, 18))

# Create a heatmap using Seaborn to visualize the filtered correlation matrix
# 'annot=True' adds the correlation values to the cells for better readability
# 'cmap='coolwarm'' specifies the color map to be used, with warm colors (red) for positive correlations
# and cool colors (blue) for negative correlations
# 'linewidths' sets the width of the white lines around the squares
sns.heatmap(corrMatrix_filtered,
            annot=True,
            cmap='coolwarm',
            linewidths=0.5);

No Collineartiy obove the 80%. We do not drop anything here. 

## Feature selection based on features and labels

###  K Best

In [345]:
X_num_train = X_train.select_dtypes(include="number").copy()
y_num_train = y_train


# Run the univariate (= one feature at a time) linear regression tests.
f_stat, _ = f_regression(X_num_train, y_num_train)

# Create a DataFrame with the f-statistics and p-values.
f_test = pd.DataFrame({"f_stat":f_stat},
                      index=X_num_train.columns)

# Sort the values in the f-statistic column.
f_test.sort_values("f_stat", ascending=False)

ValueError: Input X contains NaN.

# Models

## Decisiontree

In [237]:
from sklearn.model_selection import GridSearchCV

full_pipeline = make_pipeline(preprocessor,
                              DecisionTreeClassifier())

'''param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean","median"],
    "decisiontreeclassifier__max_depth": range(1, 21),
    "decisiontreeclassifier__min_samples_leaf": range(1, 21)
}'''

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean"],
    "decisiontreeclassifier__max_depth": [18],
    "decisiontreeclassifier__min_samples_leaf": [10]
}

search = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      verbose=1)

search.fit(X_train, y_train)

scores = {"dtree" : search.best_score_}
print("Best parameters: ", search.best_params_)

scores

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/Users/Smirnov/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best parameters:  {'columntransformer__num_pipe__simpleimputer__strategy': 'mean', 'decisiontreeclassifier__max_depth': 18, 'decisiontreeclassifier__min_samples_leaf': 10}


{'dtree': 0.014555592237995671}

## Decisiontree - Analysis

In [238]:
# Get predictions for the training and testing datasets
y_train_pred = search.predict(X_train)
y_test_pred = search.predict(X_test)

# Create confusion matrices
confusion_matrix_train = confusion_matrix(y_train, y_train_pred)
confusion_matrix_test = confusion_matrix(y_test, y_test_pred)

# Calculate accuracy for training and testing datasets
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", round(accuracy_train,3))
print("Testing Accuracy:", round(accuracy_test,3),"\n")


dtree_r2 = r2_score(y_true = y_test,
                    y_pred = y_test_pred)

print("R-squared:", round(dtree_r2,3))

dtree_r2 = mean_absolute_percentage_error(y_true = y_test,
                    y_pred = y_test_pred)

print("MAPE:", round(dtree_r2,3))

Training Accuracy: 0.17
Testing Accuracy: 0.003 

R-squared: 0.372
MAPE: 0.202


## KNN


In [239]:
# Modeling Pipe - 2

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier


full_pipeline = make_pipeline(preprocessor,
                              KNeighborsClassifier())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean"],
    "kneighborsclassifier__n_neighbors": [1]
}

search2 = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=10,
                      verbose=1)

search2.fit(X_train, y_train)

scores2 = {"knn" : search2.best_score_}
print("Best parameters: ", search2.best_params_)

scores2


Fitting 10 folds for each of 1 candidates, totalling 10 fits


/Users/Smirnov/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


Best parameters:  {'columntransformer__num_pipe__simpleimputer__strategy': 'mean', 'kneighborsclassifier__n_neighbors': 1}


{'knn': 0.006005010315355144}

## KNN - Analysis

In [240]:
# Get predictions for the training and testing datasets
y_train_pred = search2.predict(X_train)
y_test_pred = search2.predict(X_test)

# Create confusion matrices
confusion_matrix_train = confusion_matrix(y_train, y_train_pred)
confusion_matrix_test = confusion_matrix(y_test, y_test_pred)

# Calculate accuracy for training and testing datasets
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", round(accuracy_train,3))
print("Testing Accuracy:", round(accuracy_test,3),"\n")


dtree_r2 = r2_score(y_true = y_test,
                    y_pred = y_test_pred)

print("R-squared:", round(dtree_r2,3))

dtree_r2 = mean_absolute_percentage_error(y_true = y_test,
                    y_pred = y_test_pred)

print("MAPE:", round(dtree_r2,3))

Training Accuracy: 1.0
Testing Accuracy: 0.007 

R-squared: 0.467
MAPE: 0.205


## RandomForest

In [241]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

full_pipeline = make_pipeline(preprocessor,
                              RandomForestClassifier())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy": ["median"],
    "randomforestclassifier__n_estimators": [500],  # Adjust the number of trees
    "randomforestclassifier__max_depth": [10],  # Adjust the maximum depth of trees
    "randomforestclassifier__max_features": ["log2"],  # Try different feature subset sizes
    "randomforestclassifier__criterion": ["gini"]  # Try both Gini and entropy criteria
}


'''param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy": ["median"],
    "randomforestclassifier__n_estimators": [100, 300, 500],  # Adjust the number of trees
    "randomforestclassifier__max_depth": [None, 10, 20, 30],  # Adjust the maximum depth of trees
    "randomforestclassifier__max_features": ["sqrt", "log2"],  # Try different feature subset sizes
    "randomforestclassifier__criterion": ["gini", "entropy"]  # Try both Gini and entropy criteria
}
'''
# need to be rearange 
search3 = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      verbose=1)

search3.fit(X_train, y_train)

scores3 = {"rf" : search3.best_score_}
print("Best parameters: ", search3.best_params_)
scores3

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/Users/Smirnov/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best parameters:  {'columntransformer__num_pipe__simpleimputer__strategy': 'median', 'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__max_depth': 10, 'randomforestclassifier__max_features': 'log2', 'randomforestclassifier__n_estimators': 500}


{'rf': 0.01370455962730641}

## RandomForest - Analysis

In [242]:
# Get predictions for the training and testing datasets
y_train_pred = search3.predict(X_train)
y_test_pred = search3.predict(X_test)

# Create confusion matrices
confusion_matrix_train = confusion_matrix(y_train, y_train_pred)
confusion_matrix_test = confusion_matrix(y_test, y_test_pred)

# Calculate accuracy for training and testing datasets
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", round(accuracy_train,3))
print("Testing Accuracy:", round(accuracy_test,3),"\n")


dtree_r2 = r2_score(y_true = y_test,
                    y_pred = y_test_pred)

print("R-squared:", round(dtree_r2,3))

dtree_r2 = mean_absolute_percentage_error(y_true = y_test,
                    y_pred = y_test_pred)

print("MAPE:", round(dtree_r2,3))

Training Accuracy: 0.988
Testing Accuracy: 0.017 

R-squared: 0.704
MAPE: 0.157


### SGDRegressor

In [336]:
from sklearn.linear_model import LinearRegression, SGDRegressor

sgd_pipeline = make_pipeline(preprocessor,
                            SGDRegressor())

sgd_pipeline.fit(X_train, y_train)

sgd_predictions = sgd_pipeline.predict(X_test)

SGD_mape = mean_absolute_percentage_error(y_true = y_test,
                    y_pred = y_test_pred)

print("MAPE:", round(SGD_mape,3))

SGR_r2 = r2_score(y_true = y_test,
                    y_pred = sgd_predictions)

print("R-squared:", round(SGR_r2, 3))

MAPE: 0.157
R-squared: 0.841


### LinearRegression

In [333]:
# Scale the data! 

lr_pipeline = make_pipeline(preprocessor,
                            LinearRegression())

lr_pipeline.fit(X_train, y_train)

lr_predictions = lr_pipeline.predict(X_test)

lr_mape = mean_absolute_percentage_error(y_true = y_test,
                    y_pred = y_test_pred)

print("MAPE:", round(lr_mape,3))

lr_r2 = r2_score(y_true = y_test,
                    y_pred = lr_predictions)

print("R-squared:", round(lr_r2,3))

MAPE: 0.157
R-squared: 0.866


# Iteration 0

In [266]:
y_test_pred_tree = search.predict(X_test)
y_test_pred_knn = search2.predict(X_test)
y_test_pred_rf = search3.predict(X_test)
y_test_pred_SGDR = sgd_pipeline.predict(X_test)
y_test_pred_lr = lr_pipeline.predict(X_test)


baseline_tree_r2 = r2_score(y_test, y_test_pred_tree)
baseline_knn_r2 = r2_score(y_test, y_test_pred_knn)
baseline_rf = r2_score(y_test, y_test_pred_rf)
baseline_SGDR = r2_score(y_test, y_test_pred_SGDR)
baseline_lr = r2_score(y_test, y_test_pred_lr)


performances = pd.DataFrame({'decision_tree': round(baseline_tree_r2,3),
                             'knn': round(baseline_knn_r2,3), 
                             'RF': round(baseline_rf,3),
                             'SGDR': round(baseline_SGDR, 2),
                            'LR': round(baseline_lr,3)},
                            index=['baseline'])

performances

,decision_tree,knn,RF,SGDR,LR
baseline,0.372,0.467,0.704,-2.353635e+24,0.884


With out Threshold and Skaler

| Model       | Decision Tree | KNN   | Random Forest | SGDR  | Logistic Regression |
|-------------|---------------|-------|---------------|-------|----------------------|
| Baseline    | 0.372         | -0.229 | 0.697         | -3.505539e+22  | 0.884               |

With Threshold and Skaler

| Model       | Decision Tree | KNN   | Random Forest | SGDR  | Logistic Regression |
|-------------|---------------|-------|---------------|-------|----------------------|
| Baseline    | 0.454         | 0.108 | 0.595         | 0.84  | 0.866                |


With Threshold and Skaler and GridSearh

| Model       | Decision Tree | KNN   | Random Forest | SGDR  | Logistic Regression |
|-------------|---------------|-------|---------------|-------|----------------------|
| Baseline    | 0.31           | 0.64 | 0.7           | 0.87  |   -7.169538e+19          |

In [246]:
print('Readoy')

Readoy


# Downloand

In [337]:
X_sumbmition = test_housing_data
ids_com['SalePrice'] = sgd_pipeline.predict(X_sumbmition) # only cange the piplene! 
ids_com.to_csv(r'submission_sgd.csv', index=False) #only cahnge ist